# Network elements by country

This notebook generates for each country of the European model a list of generators and loads (not lines, because they can belong to two countries at the same time).

Start with importing the model:

In [1]:
import json
import pandas as pd

In [2]:
with open('../europe.json') as f:
    model = json.load(f)

List of countries in the European model:

In [3]:
countries = {bus['country'] for bus in model['bus'].values()} - {'XX'}
countries

{'AL',
 'AT',
 'BA',
 'BE',
 'BG',
 'CH',
 'CZ',
 'DE',
 'DK',
 'ES',
 'FR',
 'GR',
 'HR',
 'HU',
 'IT',
 'LU',
 'ME',
 'MK',
 'NL',
 'PL',
 'PT',
 'RO',
 'RS',
 'SI',
 'SK'}

Get the (ordered) list of elements (generators or loads) for all the countries in the list, based on the location of the bus to which they are attached:

In [4]:
sort_string_list = lambda id_list: list(map(str, sorted(map(int, id_list))))

In [5]:
def elements_by_country(element_name, bus_reference):
    list_by_country = {country: [] for country in countries}
    for id, element in model[element_name].items():
        country = model['bus'][str(element[bus_reference])]['country']
        if country in countries:
            list_by_country[country].append(id)
    return {country: sort_string_list(ids) for country, ids in list_by_country.items()}

In [6]:
gens_by_country = elements_by_country('gen', 'gen_bus')
loads_by_country = elements_by_country('load', 'load_bus')

Some countries have very few generators and loads:

In [7]:
gens_by_country['LU'], loads_by_country['LU']

(['323', '324'],
 ['3078',
  '3079',
  '3080',
  '3081',
  '3082',
  '3083',
  '3084',
  '3085',
  '3086',
  '3087',
  '3088',
  '6008'])

Other have many:

In [8]:
len(gens_by_country['DE']), len(loads_by_country['DE'])

(101, 560)

The maximum numbers of generators and loads in a single country are:

In [9]:
n_gen = max(map(len, gens_by_country.values()))
n_loads = max(map(len, loads_by_country.values()))
n_gen, n_loads

(116, 908)

Arrange the elements into Pandas dataframes with country codes as column names
(there are empty values since not all countries have the same number of elements):

In [10]:
for list_of_gens in gens_by_country.values():
    list_of_gens += (n_gen - len(list_of_gens)) * ['']

for list_of_loads in loads_by_country.values():
    list_of_loads += (n_loads - len(list_of_loads)) * ['']

In [11]:
gens_by_country_df = pd.DataFrame(gens_by_country)
loads_by_country_df = pd.DataFrame(loads_by_country)

Export the dataframes:

In [12]:
gens_by_country_df.to_csv('../gens_by_country.csv', index=False)
loads_by_country_df.to_csv('../loads_by_country.csv', index=False)

## Usage example

The list of generator and loads of a given country (here Luxemburg) can be obtained from the CSV files as follows:

In [13]:
LU_gens = pd.read_csv('../gens_by_country.csv', usecols=['LU'], dtype=str).dropna().squeeze().to_list()
LU_gens

['323', '324']

In [14]:
LU_loads = pd.read_csv('../loads_by_country.csv', usecols=['LU'], dtype=str).dropna().squeeze().to_list()
LU_loads

['3078',
 '3079',
 '3080',
 '3081',
 '3082',
 '3083',
 '3084',
 '3085',
 '3086',
 '3087',
 '3088',
 '6008']

This can be used to access time series of that country only, for instance:

In [15]:
pd.read_csv('../../run/data/export/loads_2016_1.csv', usecols=LU_loads)

,3078,3079,3080,3081,3082,3083,3084,3085,3086,3087,3088,6008
0,0.236626,0.227582,0.239990,0.226452,0.221404,0.275073,0.165748,0.232170,0.241373,0.223718,0.305888,0.264032
1,0.226668,0.229976,0.229021,0.226951,0.221270,0.256042,0.162583,0.229292,0.247517,0.219439,0.272432,0.245352
2,0.223716,0.220527,0.237047,0.226941,0.221506,0.258780,0.160903,0.224551,0.241325,0.223325,0.279277,0.246493
3,0.225039,0.227721,0.237464,0.231941,0.227200,0.253568,0.161898,0.229732,0.247956,0.226362,0.265503,0.241450
4,0.238377,0.238667,0.255946,0.242999,0.241383,0.265073,0.165119,0.243900,0.253484,0.241228,0.278720,0.255886
...,...,...,...,...,...,...,...,...,...,...,...,...
8731,0.280524,0.290300,0.282380,0.278261,0.282613,0.292527,0.186947,0.296133,0.291739,0.278268,0.303053,0.308819
8732,0.265416,0.272582,0.265762,0.262973,0.265737,0.277895,0.179779,0.279144,0.274937,0.264009,0.291586,0.296613
8733,0.272061,0.289709,0.272048,0.269397,0.272503,0.278038,0.177280,0.284267,0.291118,0.258719,0.270404,0.263645
8734,0.257849,0.281958,0.265042,0.274093,0.269990,0.271905,0.184763,0.273649,0.298844,0.256997,0.251633,0.238946


In [16]:
pd.read_csv('../../run/data/export/gens_2016_1.csv', usecols=LU_gens)

,323,324
0,0.000000,0.0
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.055855,0.0
...,...,...
8731,2.853955,0.0
8732,2.454273,0.0
8733,2.361123,0.0
8734,1.964938,0.0
